In [ ]:
'''Importing Modules'''

'''
Prerequisites:
NumPy             https://numpy.org/doc/stable/
Matplotlib        https://matplotlib.org/stable/index.html
PyTorch           https://pytorch.org/docs/stable/index.html
Torchvision       https://pytorch.org/docs/stable/index.html
PIL               https://pillow.readthedocs.io/en/stable/
GitPython         https://gitpython.readthedocs.io/en/stable/
split-folders:    https://pypi.org/project/split-folders/
python-dotenv:    https://pypi.org/project/python-dotenv/
'''

# vanilla:
import os
import json
import time
from time import strptime
import datetime
from datetime import timedelta
import shutil
from collections import OrderedDict
import random
import sys
# external:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import datasets, transforms, models
import PIL.Image as Image
import git
import splitfolders
from dotenv import load_dotenv
# custom:
from routines import *
from displays import *
import myTransforms

println([('done.', 'g')])

In [ ]:
'''Parsing the Directory Supplied by User'''

